In [2]:
from glob import glob as gg
#
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rc as rc
#
from netCDF4 import Dataset  
#
from datetime import datetime
from datetime import date

In [3]:
#
# load discretized mesh data
#
class Mesh:
    def __init__(self):
        self.read_mesh()

    def read_mesh(self):
        path = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/grid/'
        filename = path + 'REcoM1D_mesh.nc'
        ncid = Dataset(filename, "r", format="NETCDF4")
        
        # read dates
        self.dates = ncid.variables['dates'][:]
        
        # read coordinates
        self.longitude, self.latitude = ncid.variables['longitude'][:], ncid.variables['latitude'][:]
        
        # first date of simulation after spinup
        ind1, ind2 = np.where(self.longitude==self.longitude[0])[0], np.where(self.latitude==self.latitude[0])[0]
        index = list(set(ind1) & set(ind2))[-1]
        self.start_date = np.floor(self.dates[index])
        self.end_date = np.floor(self.dates[-1])
        
        # read depth and levels
        self.zbar, self.Z, self.nlevels = ncid.variables['zbar'][:], ncid.variables['Z'][:], ncid.variables['nlevels'][:]
        
        ncid.close()
#
mesh = Mesh()

In [8]:
class atm_forcing:
    def __init__(self, filename, mesh):
        self.read_atm_forcing(filename, mesh)
        
    def read_atm_forcing(self, filename, mesh):
        # read wind
        ncid = Dataset(filename, "r", format="NETCDF4") 
        dates = ncid.variables['calendar_date'][:]
        uatm, vatm = ncid.variables['uatm'][:], ncid.variables['vatm'][:]
        ncid.close()
        
        # crop to simulation dates
        ind1, ind2 = np.where(dates>=np.min(mesh.dates))[0], np.where(dates<=np.max(mesh.dates))[0]
        indices = list(set(ind1) & set(ind2))
        self.dates, self.uatm, self.vatm = dates[indices], uatm[indices], vatm[indices]
        
#
# read atm forcing
path = '/home/fbirrien/NuArctic/nuarctic/Data_Processing/data/'
filename = path + 'MOSAiC_atm_forcing'
        
forcing = atm_forcing(filename,mesh)